In [2]:
from langgraph.graph import StateGraph ,END,START
from typing import TypedDict ,Annotated,List
from langchain_ollama import ChatOllama
from langgraph.checkpoint.memory import InMemorySaver

In [ ]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(
    model="phi3:mini",
    temperature=0
)


In [19]:
class jokestate(TypedDict):
    topic:str
    joke:str
    explanation:str
    

In [13]:
def generate_joke(state:jokestate):
    topic=state['topic']
    prompt=f'generate the joke on the topic {topic}'
    response=llm.invoke(prompt).content

    return {'joke':response}

In [15]:
def generate_explanation(state:jokestate):
    joke=state['joke']
    prompt=f'generate the  explanation for the joke {joke}'
    response=llm.invoke(prompt).content

    return {'explanation':response}

In [16]:
graph=StateGraph(jokestate)
graph.add_node('generate_joke',generate_joke)
graph.add_node('generate_explanation',generate_explanation)

graph.add_edge(START , 'generate_joke')
graph.add_edge('generate_joke' ,'generate_explanation')
graph.add_edge('generate_explanation',END)

checkpointer=InMemorySaver()

workflow=graph.compile(checkpointer=checkpointer)

In [17]:
config={"configurable":{"thread_id":"1"}}
workflow.invoke({'topic':'pizza'} ,config=config)

{'topic': 'pizza',
 'joke': "Why don't scientists trust atoms when it comes to making a good pizza? Because they can never make up their minds if they want them in or out of the dough! (Pun intended, because even quantum physics has its own take on this joke!)",
 'explanation': 'The joke "Why don\'t scientists trust atoms when it comes to making a good pizza? Because they can never make up their minds if they want them in or out of the dough!" is based on wordplay and scientific concepts. Atoms are fundamental building blocks that form molecules, which then combine to create various substances including ingredients for food like cheese (made from milk proteins) and tomatoes (which contain water).\n\nThe joke plays with two ideas: the uncertainty principle in quantum physics and a common phrase about people being indecisive. In quantum mechanics, particularly Heisenberg\'s Uncertainty Principle, it is impossible to simultaneously know both the exact position and momentum of an electron 